# Part 3. DEAP Dataset + Asymmetry + SVM

In this part 3, we will focus on feature engineering using asymmetry analysis.  Asymmetry analysis here refers to the analysis of imbalance between left and right symmetrical location.

Asymmetry analysis is another very basic and must-do analysis for emotions/cognitions/resting state.

In this part, we shall extract these asymmetries as features and then input these features into SVM and see if these features are useful for predicting the four valence-arousal classes that we have obtained from Part 1.

In [1]:
import numpy as np

## 1. Loading dataset

In [2]:
# this time I will load the Dataset class from `./components/dataset.py`
from components.dataset import Dataset
path = "data"  #create a folder "data", and inside put s01.dat,....,s32.dat inside from the preprocessed folder from the DEAP dataset
dataset = Dataset(path, "Valence")

data  = np.array(dataset[:]['data'])
label = np.array(dataset[:]['label']).squeeze()

print("Data shape: " , data.shape)  #15360 = 32 * 40 trials * 12 segments, 32 EEG channels, 672 samples
print("Label shape: ", label.shape)  #two classes of valence

Data shape:  (15360, 32, 672)
Label shape:  (15360,)


## 2. Asymmetry Analysis

Before we jump into the analysis, let's us think what did we do in the previous part2.

First, we extract PSD in `3` different ways.

- way_1. X_65 => We see spectral of the entire head the same
- way_2. X_5 => We kind-of summarize the X_65 into 5 different bands but still a capture of the entire haed
- way_3. (_,n_channels * freq_bands) => We look at each band on each locations of the head (channels). This data is more spatial than the previous ones.

Finally, we learnt that more spatial data causes a better classification accuracy.

This **might be** that frequency bands at different location and time is related to emotion

WOW~!! sugoi!!!

But we did extract the features that way (_, n_channels * freq_bands), if what I have said is true then there is nothing else we could do. right?

Wellllllllllll............. There is a way.

That why you are reading this tutorial.


INTRODUCING!!!!! 

### ASYMMETRY ANALYSIS!!!


Okay, sorry. I quite playing now.

The name sounds fancy but actually the idea is simple. If you are happy, Alpha band on the left side of the head is higher than the right. (This is not true because I can not remember which side is higher but you get the idea).

To help SVM capture this behavior, we will explicitly include this information as the feature. (You know, traditional ML is not every good at correlating the feature. We have to help them.)

You can compare front-back, left-right, up-down, Fp1-Oz, .... The possibility is endless. Make sure to review papers to scope down the idea Or just run all of them (if you have time).

Okay, enough chitchat. Let's do this!!!

Here is the channels and their index.

- Channels: 32
  1.	Fp1	
  2.	AF3
  3.	F3
  4.	F7
  5.	FC5
  6.	FC1
  7.	C3
  8.	T7
  9.	CP5
  10.	CP1
  11.	P3
  12.	P7
  13.	PO3
  14.	O1
  15.	Oz
  16.	Pz
  17.	Fp2
  18.	AF4
  19.	Fz
  20.	F4
  21.	F8
  22.	FC6
  23.	FC2
  24.	Cz
  25.	C4
  26.	T8
  27.	CP6
  28.	CP2
  29.	P4
  30.	P8
  31.	PO4
  32.	O2

<img src="https://upload.wikimedia.org/wikipedia/commons/6/6e/International_10-20_system_for_EEG-MCN.svg">


I will seperate the channels into left-ones and right-ones

Based on 10-20 system, the followed number if odd, is on the left (if even, on the right). `z` is middle.

[EEG-Based Emotion Recognition Using Logistic Regression with Gaussian Kernel and Laplacian Prior and Investigation of Critical Frequency Bands](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwj7kei-1Mr2AhVTzDgGHdH_D3oQFnoECAkQAQ&url=https%3A%2F%2Fwww.mdpi.com%2F2076-3417%2F10%2F5%2F1619%2Fpdf&usg=AOvVaw3nwBT1NPFALmqKKm4rbIuE)

Based on this paper, we there are 14 pairs of left-right of asymmetry we can engineer (fancy word for calculate).

1. Fp1-Fp2
2. F7-F8
3. F3-F4
4. T7-T8
5. P7-P8
6. C3-C4
7. P3-P4
8. O1-O2
9. AF3-AF4
10. FC5-FC6
11. FC1-FC2
12. CP5-CP6
13. CP1-CP2
14. PO3-PO4

And there are two relation we can create. Differential Asymmetry (DASM) and Rational Asymmetry (RASM)

Another 11 pairs of frontal-posterior is as follow

1. FC5-CP5
2. FC1-CP1
3. FC2-CP2
4. FC6-CP6
5. F7-P7
6. F3-P3
7. Fz-Pz
8. F4-P4
9. F8-P8
10. Fp1-O1
11. Fp2-O2

The paper uses differential caudality (DCAU)

$$ DASM = DE(X_{left}) - DE(X_{right}) $$
$$ RASM = \frac{DE(X_{left})}{DE(X_{right})} $$
$$ DCAU = DE(X_{frontal}) - DE(X_{posterior}) $$

$ DE() $ is a function that convert PSD to log-PSD. (There is more about it please read the paper)

In [3]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")

left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


Let's get PSD. I will use `MNE_feature` since it handles epochs-like data for me and I don't need to analyze the data (remember, the data is already preprocessed)

The idea is as follow;

1. I will slowly calculate PSD of each frequncy band.
2. For each band, I will calculate DASM, RASM, and DCAU

In [6]:
from mne_features.feature_extraction import FeatureExtractor

bands = dict({
    "delta": (0,4), 
    "theta": (4,8),
    "alpha": (8,12),
     "beta": (12,30),
    "gamma": (30,64)
})

PSDs = []
DASMs = []
RASMs = []
DCAUs = []
for band_name,band_range in bands.items():
    params = dict({
        'pow_freq_bands__log':True,
        'pow_freq_bands__normalize':False,
        'pow_freq_bands__freq_bands':band_range
    })
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8, memory="cache/")
    PSD = fe.fit_transform(X=data)
    #      (15360, 32)
    # print(PSD.shape)
    

    PSD_left = PSD[:, left_channel_indexes].copy()
    PSD_right = PSD[:, right_channel_indexes].copy()
    PSD_frontal = PSD[:, frontal_channel_indexes].copy()
    PSD_posterior = PSD[:, posterior_channel_indexes].copy()
    #       (15360, 14)     (15360, 14)      (15360, 11)        (15360, 11)
    # print(PSD_left.shape, PSD_right.shape, PSD_frontal.shape, PSD_posterior.shape)
    DASM = PSD_left - PSD_right
    #      (15360, 14)
    # print(DASM.shape)
    RASM = PSD_left / PSD_right
    #      (15360, 14)
    # print(RASM.shape)
    DCAU = PSD_frontal - PSD_posterior
    #      (15360, 11)
    # print(DCAU.shape)

    # Will be use later for comparison
    PSDs.append(PSD)
    DASMs.append(DASM)
    RASMs.append(RASM)
    DCAUs.append(DCAU)

PSDs = np.hstack(PSDs)
DASMs = np.hstack(DASMs)
RASMs = np.hstack(RASMs)
DCAUs = np.hstack(DCAUs)

#     (15360, 160) (15360, 70) (15360, 70) (15360, 55)
print(PSDs.shape, DASMs.shape, RASMs.shape, DCAUs.shape)

(15360, 160) (15360, 70) (15360, 70) (15360, 55)


## 3. Machine Learning

In [7]:
y = label.squeeze()
y.shape

(15360,)

In [8]:
def train_model(X_ori,y_ori, kernel='rbf'):
    # Make a copy because I am paranoid
    X,y = X_ori.copy(), y_ori.copy()

    from sklearn.svm import SVC
    from sklearn.utils import shuffle
    from sklearn.model_selection import cross_val_score

    X_shuff,y_shuff = shuffle(X,y)
    model = SVC(kernel=kernel,max_iter=10000)
    cross = cross_val_score(model, X_shuff, y_shuff, cv=3)

    model = SVC(kernel=kernel, max_iter=10000)
    model.fit(X_shuff, y_shuff)
    ans = model.predict(X_shuff)
    acc = sum(ans == y_shuff) / len(y_shuff)
    return model, acc, cross

In [9]:
import time
fe_name = ['PSDs', 'DASMs', 'RASMs', 'DCAUs']
for kernel in ['linear','poly','rbf', 'sigmoid']:
    for index, X in enumerate([PSDs, DASMs, RASMs, DCAUs]):
        start = time.time()
        model, acc, cross = train_model(X, y, kernel=kernel)
        # We can save the model and reuse it later
        print(f"\tKernel={kernel}-{fe_name[index]}| Acc={round(acc,5)} | 3-CV score={round(cross.mean(),5)} STD={round(cross.std(),5)}| Time spend={time.time() - start}")

/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=linear-PSDs| Acc=0.53633 | 3-CV score=0.53288 STD=0.02082| Time spend=58.18406867980957


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=linear-DASMs| Acc=0.55469 | 3-CV score=0.55527 STD=0.01589| Time spend=32.90813899040222


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


...

	Kernel=poly-DASMs| Acc=0.72682 | 3-CV score=0.68158 STD=0.01047| Time spend=35.642327547073364


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=poly-RASMs| Acc=0.55365 | 3-CV score=0.55299 STD=9e-05| Time spend=38.68176770210266


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=poly-DCAUs| Acc=0.73197 | 3-CV score=0.69564 STD=0.00779| Time spend=31.705512285232544
	Kernel=rbf-PSDs| Acc=0.7043 | 3-CV score=0.67533 STD=0.00507| Time spend=117.91553449630737
	Kernel=rbf-DASMs| Acc=0.73314 | 3-CV score=0.69824 STD=0.0039| Time spend=57.48490333557129
	Kernel=rbf-RASMs| Acc=0.55358 | 3-CV score=0.55319 STD=9e-05| Time spend=68.62462854385376
	Kernel=rbf-DCAUs| Acc=0.73151 | 3-CV score=0.69258 STD=0.00523| Time spend=52.47633194923401
	Kernel=sigmoid-PSDs| Acc=0.48691 | 3-CV score=0.48711 STD=0.00172| Time spend=64.99256181716919
	Kernel=sigmoid-DASMs| Acc=0.52083 | 3-CV score=0.51204 STD=0.00512| Time spend=43.00359034538269
	Kernel=sigmoid-RASMs| Acc=0.55332 | 3-CV score=0.55267 STD=0.00109| Time spend=59.13428854942322
	Kernel=sigmoid-DCAUs| Acc=0.51667 | 3-CV score=0.52572 STD=0.01493| Time spend=38.558884620666504


Oh wow. The result is pretty good.

Do you think if we use all of them, it will achieved more?

Why don't we do it now.

In [13]:
all_X = np.concatenate( [PSDs, DASMs, RASMs, DCAUs], axis=1  )
print(all_X.shape)

(15360, 355)


In [15]:
import time
for kernel in ['linear','poly','rbf', 'sigmoid']:
        start = time.time()
        model, acc, cross = train_model(all_X, y, kernel=kernel)
        # We can save the model and reuse it later
        print(f"\tKernel={kernel}-all_X| Acc={round(acc,5)} | 3-CV score={round(cross.mean(),5)} STD={round(cross.std(),5)}| Time spend={time.time() - start}")

/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=linear-all_X| Acc=0.55241 | 3-CV score=0.54941 STD=0.00967| Time spend=5.466062307357788


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=poly-all_X| Acc=0.44681 | 3-CV score=0.48236 STD=0.05018| Time spend=201.3437066078186
	Kernel=rbf-all_X| Acc=0.55365 | 3-CV score=0.55319 STD=9e-05| Time spend=269.5489151477814
	Kernel=sigmoid-all_X| Acc=0.55326 | 3-CV score=0.55384 STD=0.00088| Time spend=236.1786971092224


Uh no.

It takes longer time to train but achieved less than a smaller set of features.

Confuse?

Welcome to EEG world. Are you not entertained?